# Prediction of anxiety-related phenomena fron error-related brain signal

### Imports

In [1]:
import os
import re
import glob
import os
import ast
import os.path as op
from collections import defaultdict
from copy import deepcopy
import copy

import pickle
from time import time
import pywt
import mne
import scipy
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import cesium.featurize
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, FloatSlider, interact
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from tempfile import mkdtemp
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from mne.decoding import SPoC


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import random
import copy



from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA

from transformers import *

import warnings

warnings.filterwarnings("ignore")

import sys

---
## Load data

Loading EEG data and data from rumination questionnaire. By default create_df_data loads all info from given file but one can specify it by passing a list of desired labels from csv file.

In [2]:
# paths TODO
dir_path = os.path.dirname(os.path.abspath(""))

Constants

In [3]:
tmin, tmax = -0.1, 0.6  # Start and end of the segments

signal_frequency = 256
ERROR = 0
CORRECT = 1
random_state = 0

In [4]:
def create_df_data(
    test_participants=False,
    test_epochs=False,
    info_filename=None,
    info="all",
    personal=True,
):
    """Loads data for all participants and create DataFrame with optional additional info from given .csv file.

    On default, loads a train set: chooses only 80% of participants
    and for each of them chooses 80% of epochs.
    It will choose them deterministically.

    Participants with less than 10 epochs per condition are rejected.

    If test_participants is set to True, it will load remaining 20% of participants.
    If test_epochs is set to True, it will load remaining 20% of epochs.
    Test epochs are chronologically after train epochs,
    because it reflects real usage (first callibration and then classification).

    Parameters
    ----------
    test_participants: bool
        whether load data for training or final testing.
        If true load participants data for testing.
    test_epochs: bool
        whether load data for training or final testing.
        If true load epochs of each participants data for testing.
    info_filename: String | None
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters
    personal: bool
        whether a model will be both trained and tested on epochs from one person
        if false, person's epochs aren't split into test and train


    Returns
    -------
    go_nogo_data_df : pandas.DataFrame

    """
    print(os.path.abspath(""))
    dir_path = os.path.dirname(os.path.abspath(""))
    print(dir_path)
    header_files_glob = os.path.join(dir_path, "data/responses_100_600/*.vhdr")
    header_files = glob.glob(header_files_glob)

    header_files = sorted(header_files)
    go_nogo_data_df = pd.DataFrame()

    # cut 20% of data for testing
    h_train, h_test = train_test_split(header_files, test_size=0.3, random_state=0)
    
    print(f"train size: {len(h_train)} ; test size: {len(h_test)}")

    if test_participants:
        header_files = h_test
    else:
        header_files = h_train

    for file in header_files:
        #  load eeg data for given participant
        participant_epochs = load_epochs_from_file(file)

        # and compute participant's id from file_name
        participant_id = re.match(r".*_(\w+).*", file).group(1)

        error = participant_epochs["error_response"]._data
        correct = participant_epochs["correct_response"]._data

        # exclude those participants who have too few samples
        if len(error) < 5 or len(correct) < 5:
            # not enough data for this participant
            continue

        # construct dataframe for participant with: id|epoch_data|response_type|additional info...
        participant_df = create_df_from_epochs(
            participant_id, participant_epochs, info_filename, info
        )
        print(participant_id)
        go_nogo_data_df = go_nogo_data_df.append(participant_df, ignore_index=True)

    return go_nogo_data_df

In [5]:
def create_df_from_epochs(id, participant_epochs, info_filename, info):
    """Create df for each participant. DF structure is like: {id: String ; epoch: epoch_data ; marker: 1.0|0.0}
    1.0 means correct and 0.0 means error response.
    Default info extracted form .csv file is 'Rumination Full Scale' and participants' ids.
    With this info df structure is like:
    {id: String ; epoch: epoch_data ; marker: 1.0|0.0 ; File: id ; 'Rumination Full Scale': int}

    Parameters
    ----------
    id: String
        participant's id extracted from filename
    correct: array
        correct responses' data
    error: array
        error responses' data
    info_filename: String
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters

    Returns
    -------
    participant_df : pandas.DataFrame

    """
    participant_df = pd.DataFrame()
    info_df = pd.DataFrame()

    # get additional info from file
    if info_filename is not None:
        if info == "all":
            rumination_df = pd.read_csv(info_filename)
        else:
            rumination_df = pd.read_csv(info_filename, usecols=["Demo_kod"] + info)
        info_df = (
            rumination_df.loc[rumination_df["Demo_kod"] == id]
            .reset_index()
            .drop("index", axis=1)
        )
        
    epoch_df = pd.DataFrame({"id": [id], "epoch": [participant_epochs], "marker": [ALL]}).join(
            info_df
        )
    participant_df = participant_df.append(epoch_df, ignore_index=True)

    return participant_df

In [6]:
def load_epochs_from_file(file, reject_bad_segments="auto", mask=None):
    """Load epochs from a header file.

    Args:
        file: path to a header file (.vhdr)
        reject_bad_segments: 'auto' means that bad segments are rejected automatically.

    Returns:
        mne Epochs

    """
    # Import the BrainVision data into an MNE Raw object
    raw = mne.io.read_raw_brainvision(file)

    # Construct annotation filename
    annot_file = file[:-4] + "vmrk"

    # Read in the event information as MNE annotations
    annotations = mne.read_annotations(annot_file)

    # Add the annotations to our raw object so we can use them with the data
    raw.set_annotations(annotations)

    # Map with response markers only
    event_dict = {
        "Stimulus/RE*ex*1_n*1_c_1*R*FB": 10004,
        "Stimulus/RE*ex*1_n*1_c_1*R*FG": 10005,
        "Stimulus/RE*ex*1_n*1_c_2*R": 10006,
        "Stimulus/RE*ex*1_n*2_c_1*R": 10007,
        "Stimulus/RE*ex*2_n*1_c_1*R": 10008,
        "Stimulus/RE*ex*2_n*2_c_1*R*FB": 10009,
        "Stimulus/RE*ex*2_n*2_c_1*R*FG": 10010,
        "Stimulus/RE*ex*2_n*2_c_2*R": 10011,
    }

    # Map for merged correct/error response markers
    merged_event_dict = {"correct_response": 0, "error_response": 1}

    # Reconstruct the original events from Raw object
    events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

    # Merge correct/error response events
    merged_events = mne.merge_events(
        events,
        [10004, 10005, 10009, 10010],
        merged_event_dict["correct_response"],
        replace_events=True,
    )
    merged_events = mne.merge_events(
        merged_events,
        [10006, 10007, 10008, 10011],
        merged_event_dict["error_response"],
        replace_events=True,
    )

    epochs = []
    bads = []
    this_reject_by_annotation = True
    
    # maximum acceptable peak-to-peak amplitudes
    reject_criteria = dict(eeg=150e-6)       # 200 µV
    
    # minimum acceptable peak-to-peak amplitudes
    flat_criteria = dict(eeg=1e-6)           # 1 µV
    
    picks_eeg = mne.pick_types(raw.info, meg=False, eeg=True, eog=False,
                           stim=False, exclude='bads', selection=red_box7_prim)

    # Read epochs
    epochs = mne.Epochs(
        raw=raw,
        events=merged_events,
        event_id=merged_event_dict,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        reject_by_annotation=this_reject_by_annotation,
        preload=True,
        # verbose='CRITICAL',
    )
    
    return epochs

#### Read the training data

In [7]:
df_name = "go_nogo_100_600_df_3-5_all_scales"
pickled_data_filename = "../data/responses_100_600_pickled/" + df_name + ".pkl"
info_filename = "../data/scales/all_scales.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df = pd.read_pickle(pickled_data_filename)
    print("Done")
    pass
else:
    print("Pickled file not found. Loading data...")
    epochs_df = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_df.name = df_name
    # save loaded data into a pickle file
    epochs_df.to_pickle("../data/" + epochs_df.name + ".pkl")
    print("Done. Pickle file created")

Pickled file found. Loading pickled data...
Done


#### Read the testing data

In [8]:
df_name = "go_nogo_100_600_test_df_3-5_all_scales"
pickled_data_filename = "../data/responses_100_600_pickled/" + df_name + ".pkl"
info_filename = "../data/scales/all_scales.csv"


# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_test_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_test_df = create_df_data(
        test_participants=True, info="all", personal=False, info_filename=info_filename
    )
    epochs_test_df.name = df_name
    # save loaded data into a pickle file
    epochs_test_df.to_pickle("../data/responses_100_600_pickled/" + epochs_test_df.name + ".pkl")
    print("Done. Pickle file created")

Pickled file found. Loading pickled data...
Done


## Set-up for experiments

### Prepare data

#### Create X train and y train sets

In [9]:
# selection of the analysed condition: erroneous responses or correct responses
dataset = ERROR

dataset_name = "correct_response" if dataset == CORRECT else "error_response"

In [10]:
X_train_df = epochs_df
X_test_df = epochs_test_df

print(f"Shape of the training set: {X_train_df.shape}")
print(f"Shape of the testing set: {X_test_df.shape}")

Shape of the training set: (96, 163)
Shape of the testing set: (34, 163)


#### Fill NaNs

In [11]:
X_train_df = X_train_df.fillna(X_train_df.mean())

X_train_df.isnull().sum()

id                                                                                                               0
epoch                                                                                                            0
marker                                                                                                           0
Demo_kod                                                                                                         1
Pula badań                                                                                                       0
                                                                                                                ..
35-IN-2.\tJaką część czasu spędzonego on-line poświęcasz na pracę lub naukę?                                     0
35-IN-3.\tJaką część czasu spędzonego on-line poświęcasz na media społecznościowe?                               0
35-IN-4.\tJaką część czasu spędzonego on-line poświęcasz na granie w gry kompute

#### Select questionnaires for modelling

In [12]:
# shape 1-D: scores
rumination = "16-Rumination Full Scale"
dass_anxiety = "05-DASS-21 Anxiety scale"
stai_t = "04-STAI Trait SUM" 
bis = "07-BIS"
bas_dzialanie = "07-BAS Dzialanie"
bas_przyjemnosc = "07-BAS Poszukiwanie przyjemnosci"
bas_nagroda = "07-BAS Wrazliwosc na nagrode"
washing = "14-Obsessive-Compulsive WASHING"
obsessing = "14-Obsessive-Compulsive OBSESSING"
hoarding = "14-Obsessive-Compulsive HOARDING"
ordering = "14-Obsessive-Compulsive ORDERING"
checking = "14-Obsessive-Compulsive CHECKING"
neutralizing = "14-Obsessive-Compulsive NEUTRALIZING"
oci_r_full = "14-Obsessive-Compulsive FULL"
threat = "15-Obsessional Beliefs - Overestimation of threat"
perfectionism_IU = "15-Obsessional Beliefs - Perfectionism/ Intolerance of uncertainty"
thought_suppression = "18-Thought Suppression Inventory"
nonforgivness = "22-Nonforgiveness - Full Scale"
indecisivness = "27-Indecisiveness Scale_Frost"
IU_prospecitve = "28-Intolerance of Uncertainty - Prospective Anxiety"
IU_inhibitory = "28-Intolerance of Uncertainty - Inhibitory Anxiety"
self_esteem = "06-Self-Esteem Scale_SES Rosenberga"

In [13]:
scales = [
    rumination,
    dass_anxiety,
    stai_t,
    bis,
    washing,
    obsessing,
    hoarding,
    ordering,
    checking,
    neutralizing,
    oci_r_full,
    threat,
    thought_suppression,
    IU_prospecitve,
    IU_inhibitory,
    self_esteem,
]

### Prepare experiments 

Parameters of experiments:
- regressors
- hyperparameters
- preprocessing pipelines

In [14]:
# Rating model with grid search

def rate_regressor(
    X_train, 
    y_train, 
    X_test, 
    y_test, 
    regressor, 
    regressor_params, 
    base_steps, 
    cv=3
):
    
    # define cross-validation method
    cv_kf = KFold(n_splits=3)

    pipeline = Pipeline(base_steps + [regressor])
    param_grid = regressor_params
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv_kf,
        scoring={"r2", "neg_mean_absolute_error", "neg_mean_squared_error"},
        refit="r2",
        return_train_score=True,
        n_jobs=10,
        verbose=10,
    )
    
    grid_search.fit(X_train, y_train)

    return grid_search

In [15]:
# Calculating p-value with permutation test from sci-kit learn

def calculate_p_permutations(estimator, X, y, cv=3, n_permutations=1000, n_jobs=10):

    score_, perm_scores_, pvalue_ = permutation_test_score(
        estimator, X, y, cv=cv, n_permutations=n_permutations, n_jobs=n_jobs
    )

    # summarize
    print(f"     The permutation P-value is = {pvalue_:.4f}")
    print(f"     The permutation score is = {score_:.4f}\n")

    return score_, pvalue_

In [16]:
# self-implemented permutation test for testing significance of tests on hold-out dataset

def permutation_test_score_external(
    estimator,
    X,
    y,
    *,
    groups=None,
    cv=None,
    n_permutations=100,
    n_jobs=None,
    random_state=0,
    verbose=0,
    scoring=None,
    fit_params=None,
):
    score = estimator.score(X, y)
    print(f"score: {score} ")
    
    permutation_scores = [
        _permutation_test_score(
            estimator,
            X,
            _shuffle(y,i),
            fit_params=fit_params,
        )
        for i in range(n_permutations)]
    
    permutation_scores = np.array(permutation_scores)
    pvalue = (np.sum(permutation_scores >= score) + 1.0) / (n_permutations + 1)
    return score, permutation_scores, pvalue


def _permutation_test_score(estimator, X, y, fit_params):
    score = estimator.score(X,y)
    return score


def _shuffle(y, seed):
    random.Random(seed).shuffle(y)
    return y

In [17]:
# conducting experiment and saving selected info do result df

def run_experiment(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    preprocessed_pipeline,
    X_test_df,
    scale,
    results_df,
):

    for (regressor, params) in tested_regressors:
        print(f"Rating {regressor} \n")
        tested_params = {**regressor_params, **params}

        # enter to grid search
        grid_result = rate_regressor(
            X_train,
            y_train,
            X_test,
            y_test,
            regressor,
            tested_params,
            base_steps,
            cv=3,
        )

        best_estimator_index = grid_result.best_index_
        mean_cv_r2 = grid_result.cv_results_["mean_test_r2"][best_estimator_index]
        std_cv_r2 = grid_result.cv_results_["std_test_r2"][best_estimator_index]
        mean_cv_neg_mean_absolute_error = grid_result.cv_results_[
            "mean_test_neg_mean_absolute_error"
        ][best_estimator_index]
        std_cv_neg_mean_absolute_error = grid_result.cv_results_[
            "std_test_neg_mean_absolute_error"
        ][best_estimator_index]
        mean_cv_neg_mean_squared_error = grid_result.cv_results_[
            "mean_test_neg_mean_squared_error"
        ][best_estimator_index]
        std_cv_neg_mean_squared_error = grid_result.cv_results_[
            "std_test_neg_mean_squared_error"
        ][best_estimator_index]
        
        mean_train_r2 = grid_result.cv_results_["mean_train_r2"][best_estimator_index]
        mean_train_mae = grid_result.cv_results_["mean_train_neg_mean_absolute_error"][best_estimator_index]
        mean_train_mse = grid_result.cv_results_["mean_train_neg_mean_squared_error"][best_estimator_index]
       

        print(f"     Best parameters: {grid_result.best_params_}")
        print(f"     mean r2: {mean_cv_r2}           ± {round(std_cv_r2,3)}")
        print(f"     mean r2 train: {mean_train_r2}")

        cv_results = grid_result.cv_results_

        # calculate p-value
        scores_, pvalue_ = calculate_p_permutations(
            grid_result.best_estimator_, X_train, y_train, cv=3
        )
        
        pre_processed_test_X = preprocessed_pipeline.transform(X_test_df)
        estimator = grid_result.best_estimator_
        score = estimator.score(pre_processed_test_X, y_test)
        
        # permutation test for external
        pre_processed_test_X_copy = copy.deepcopy(pre_processed_test_X)
        y_test_copy = copy.deepcopy(y_test)

        score_ext, permutation_scores, pvalue_ext = permutation_test_score_external(estimator,
            pre_processed_test_X_copy, y_test_copy, n_permutations=1000
        )
        
        print(print(f"     external validation r2: {score}      p-value:{pvalue_ext}"))
        

        # insert selected info to df
        data = {
            "data_set": dataset_name,
            "pipeline_name": pipeline_name,
            "model": regressor[0],
            "parameters": grid_result.best_params_,
            "mean_cv_r2": mean_cv_r2,
            "std_cv_r2": std_cv_r2,
            "mean_cv_mae": mean_cv_neg_mean_absolute_error,
            "std_cv_mae": std_cv_neg_mean_absolute_error,
            "mean_cv_mse":mean_cv_neg_mean_squared_error,
            "std_cv_mse": std_cv_neg_mean_squared_error,
            "cv_results": cv_results,
            "mean_train_r2": mean_train_r2,
            "mean_train_mae":mean_train_mae,
            "mean_train_mse":mean_train_mse,
            "p-value": pvalue_,
            "best_estimator": grid_result.best_estimator_,
            "pre_processed_pipeline": preprocessed_pipeline,
            "scale": scale,
            "external_score":score,
            "external_p-value":pvalue_ext,
        }

        results_df = results_df.append(data, ignore_index=True)
    return results_df

In [18]:
# conducting experiment and saving selected info do result df. 
# Additionally estimate p-value of model's performance on full training set (no CV)

def run_experiment_estimate_train_p(
    tested_regressors,
    regressor_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    preprocessed_pipeline,
    X_test_df,
    scale,
    results_df,
):

    for (regressor, params) in tested_regressors:
        print(f"Rating {regressor} \n")
        tested_params = {**regressor_params, **params}

        # enter to grid search
        grid_result = rate_regressor(
            X_train,
            y_train,
            X_test,
            y_test,
            regressor,
            tested_params,
            base_steps,
            cv=3,
        )

        best_estimator_index = grid_result.best_index_
        mean_cv_r2 = grid_result.cv_results_["mean_test_r2"][best_estimator_index]
        std_cv_r2 = grid_result.cv_results_["std_test_r2"][best_estimator_index]
        mean_cv_neg_mean_absolute_error = grid_result.cv_results_[
            "mean_test_neg_mean_absolute_error"
        ][best_estimator_index]
        std_cv_neg_mean_absolute_error = grid_result.cv_results_[
            "std_test_neg_mean_absolute_error"
        ][best_estimator_index]
        mean_cv_neg_mean_squared_error = grid_result.cv_results_[
            "mean_test_neg_mean_squared_error"
        ][best_estimator_index]
        std_cv_neg_mean_squared_error = grid_result.cv_results_[
            "std_test_neg_mean_squared_error"
        ][best_estimator_index]
        
        mean_train_r2 = grid_result.cv_results_["mean_train_r2"][best_estimator_index]
        mean_train_mae = grid_result.cv_results_["mean_train_neg_mean_absolute_error"][best_estimator_index]
        mean_train_mse = grid_result.cv_results_["mean_train_neg_mean_squared_error"][best_estimator_index]
       

        print(f"     Best parameters: {grid_result.best_params_}")
        print(f"     mean r2: {mean_cv_r2}           ± {round(std_cv_r2,3)}")
        print(f"     mean r2 train: {mean_train_r2}")

        cv_results = grid_result.cv_results_

        # calculate p-value
        scores_, pvalue_ = calculate_p_permutations(
            grid_result.best_estimator_, X_train, y_train, cv=3
        )
        
        pre_processed_test_X = preprocessed_pipeline.transform(X_test_df)
        estimator = grid_result.best_estimator_
        score = estimator.score(pre_processed_test_X, y_test)
        
        # permutation test for external
        pre_processed_test_X_copy = copy.deepcopy(pre_processed_test_X)
        y_test_copy = copy.deepcopy(y_test)

        score_ext, permutation_scores, pvalue_ext = permutation_test_score_external(estimator,
            pre_processed_test_X_copy, y_test_copy, n_permutations=1000
        )
        
        # tests for whole train (to prove overfitting)
        score_train, permutation_scores, pvalue_train = permutation_test_score_external(estimator,
            X_train, y_train, n_permutations=1000
        )
        
        print(print(f"     external validation r2: {score}      p-value:{pvalue_ext}"))
        

        # insert selected info to df
        data = {
            "data_set": dataset_name,
            "pipeline_name": pipeline_name,
            "model": regressor[0],
            "parameters": grid_result.best_params_,
            "mean_cv_r2": mean_cv_r2,
            "std_cv_r2": std_cv_r2,
            "mean_cv_mae": mean_cv_neg_mean_absolute_error,
            "std_cv_mae": std_cv_neg_mean_absolute_error,
            "mean_cv_mse":mean_cv_neg_mean_squared_error,
            "std_cv_mse": std_cv_neg_mean_squared_error,
            "cv_results": cv_results,
            "mean_train_r2": mean_train_r2,
            "mean_train_mae":mean_train_mae,
            "mean_train_mse":mean_train_mse,
            "p-value": pvalue_,
            "best_estimator": grid_result.best_estimator_,
            "pre_processed_pipeline": preprocessed_pipeline,
            "scale": scale,
            "external_score":score,
            "external_p-value":pvalue_ext,
            "train_score": score_train,
            "train_p-value":pvalue_train,
        }

        results_df = results_df.append(data, ignore_index=True)
    return results_df

#### Global hyperparameters  of models

In [19]:
# define estimators and their hyperparameters

en = ("en", ElasticNet(random_state=random_state))
en_params = dict(
    en__alpha=np.logspace(-7, 3, num=20, base=10),
    en__l1_ratio=np.logspace(-8, 0, num=17, base=10),
)

kr = ("kr", KernelRidge(kernel="rbf"))
kr_params = dict(
    kr__alpha=np.logspace(-5, 3, num=20, base=10),
    kr__gamma=np.logspace(-5, 3, num=20, base=10),
)


svr = ("svr", SVR())
svr_params = dict(
    svr__kernel=["linear", "rbf"],
    svr__C=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
    svr__gamma=["scale"],
    svr__epsilon=[0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
)

ln = ("ln", LinearRegression())
ln_params = dict()

## Conduct experiments

### Experiment 1: Classic ERP-wave analysis for ERN

- error responses;
- average over segments per subject;
- time-window: 0 - 100 ms after response;
- channels: Fz
- feature: mean amplitude in selected time-window

In [20]:
tested_regressors = [
    (ln, ln_params),
]

regressor_params = dict()

In [21]:
results_ern_linear = pd.DataFrame()

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

this_bin=12

for scale in scales:
    print(f"---------------------SCALE: {scale}-------------------------")
    
    y_train = np.array(X_train_df[scale].to_list())
    y_test = np.array(X_test_df[scale].to_list())

    X_train_df_copy = pd.DataFrame(copy.deepcopy(X_train_df.to_dict()))
    X_test_df_copy = pd.DataFrame(copy.deepcopy(X_test_df.to_dict()))


    pipeline_name = f"ERP_scale_{scale}"

    ############################################################################################
    preprocessed_pipeline = Pipeline([
        ("channels_extraction",PickChannels(channels_list=['Fz'])),
        ("trim", EpochTrim(tmin=0, tmax=0.1)),
        ("average", Evoked()),
        ('extract_averaged_data', ExtractData()),
    ]).fit(X_train_df_copy)

    preprocessed_X = preprocessed_pipeline.transform(X_train_df_copy)


    ###########################################################################################
    features = Pipeline(steps = [
        ("mean_amplitude", AverageSignal()),
    ])

    steps = [('features', features)]

    ############################################################################################

    regressor_steps = steps

    # rate different models
    results_ern_linear = run_experiment_estimate_train_p(
        tested_regressors,
        regressor_params,
        pipeline_name,
        preprocessed_X,
        X_test_df_copy,
        y_train,
        y_test,
        dataset_name,
        regressor_steps,
        preprocessed_pipeline,
        X_test_df_copy,
        scale,
        results_ern_linear,
    )

---------------------SCALE: 16-Rumination Full Scale-------------------------
IN PE RETURN SHAPE: (206, 1, 27)
IN PE RETURN SHAPE: (206, 1, 27)
(96, 1, 27)
Rating ('ln', LinearRegression()) 

Fitting 3 folds for each of 1 candidates, totalling 3 fits
     Best parameters: {}
     mean r2: 0.016250879332354695           ± 0.024
     mean r2 train: 0.04348064433608353
     The permutation P-value is = 0.0240
     The permutation score is = 0.0163

IN PE RETURN SHAPE: (178, 1, 27)
(34, 1, 27)
score: -0.062329294047009265 
score: 0.041023834659058345 
     external validation r2: -0.062329294047009265      p-value:0.6713286713286714
None
---------------------SCALE: 05-DASS-21 Anxiety scale-------------------------
IN PE RETURN SHAPE: (206, 1, 27)
IN PE RETURN SHAPE: (206, 1, 27)
(96, 1, 27)
Rating ('ln', LinearRegression()) 

Fitting 3 folds for each of 1 candidates, totalling 3 fits
     Best parameters: {}
     mean r2: -0.0455170371465623           ± 0.032
     mean r2 train: 0.00282306

In [22]:
results_ern_linear

,data_set,pipeline_name,model,parameters,mean_cv_r2,std_cv_r2,mean_cv_mae,std_cv_mae,mean_cv_mse,std_cv_mse,...,mean_train_mae,mean_train_mse,p-value,best_estimator,pre_processed_pipeline,scale,external_score,external_p-value,train_score,train_p-value
0,error_response,ERP_scale_16-Rumination Full Scale,ln,{},0.016251,0.023665,-0.709000,0.040622,-0.786319,0.074140,...,-0.703476,-0.768525,0.023976,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",16-Rumination Full Scale,-0.062329,0.671329,0.041024,0.021978
1,error_response,ERP_scale_05-DASS-21 Anxiety scale,ln,{},-0.045517,0.031686,-0.431337,0.022361,-0.285005,0.056023,...,-0.423519,-0.274785,0.406593,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",05-DASS-21 Anxiety scale,-0.032219,0.067932,0.000088,0.495504
2,error_response,ERP_scale_04-STAI Trait SUM,ln,{},0.081966,0.075948,-7.436120,0.323006,-82.384121,2.137322,...,-7.307361,-79.303166,0.003996,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",04-STAI Trait SUM,-0.316969,0.982018,0.119894,0.000999
3,error_response,ERP_scale_07-BIS,ln,{},0.031381,0.123269,-0.475041,0.026760,-0.315684,0.022316,...,-0.466577,-0.303499,0.012987,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",07-BIS,-0.188522,0.881119,0.083292,0.001998
4,error_response,ERP_scale_14-Obsessive-Compulsive WASHING,ln,{},-0.048720,0.030112,-0.713062,0.053559,-0.751576,0.063947,...,-0.693350,-0.726523,0.457542,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",14-Obsessive-Compulsive WASHING,-0.006986,0.563437,0.001649,0.362637
5,error_response,ERP_scale_14-Obsessive-Compulsive OBSESSING,ln,{},0.017501,0.008962,-0.920692,0.024018,-1.189927,0.117030,...,-0.920861,-1.184228,0.024975,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",14-Obsessive-Compulsive OBSESSING,-0.054165,0.738262,0.024121,0.074925
6,error_response,ERP_scale_14-Obsessive-Compulsive HOARDING,ln,{},-0.007974,0.005383,-0.785336,0.148875,-0.932031,0.295094,...,-0.777371,-0.921248,0.114885,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",14-Obsessive-Compulsive HOARDING,-0.113796,0.296703,0.000268,0.428571
7,error_response,ERP_scale_14-Obsessive-Compulsive ORDERING,ln,{},-0.056413,0.049666,-1.019442,0.088147,-1.429796,0.209139,...,-0.996103,-1.377311,0.537463,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",14-Obsessive-Compulsive ORDERING,-0.008713,0.597403,0.002095,0.328671
8,error_response,ERP_scale_14-Obsessive-Compulsive CHECKING,ln,{},0.028431,0.028373,-0.871628,0.017070,-1.079984,0.011690,...,-0.861688,-1.060815,0.013986,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",14-Obsessive-Compulsive CHECKING,-0.015395,0.358641,0.044314,0.021978
9,error_response,ERP_scale_14-Obsessive-Compulsive NEUTRALIZING,ln,{},-0.074319,0.042541,-0.482264,0.051636,-0.349271,0.087268,...,-0.469215,-0.332906,0.620380,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Fz']), EpochTrim...",14-Obsessive-Compulsive NEUTRALIZING,-0.034465,0.320679,0.000460,0.400599


In [31]:
results_ern_linear.to_csv("../public_data/results/models_results/ern_waves_analysis_train_cv_lin.csv")

In [23]:
results_ern_linear.to_pickle("../public_data/results/models_pickles/ern_waves_analysis_train_cv_lin.pkl")

---
### Experiment 2: Classic ERP-wave analysis for Pe

- error responses;
- average over segments per subject;
- time-window: 150 - 350 ms after response;
- channels: Cz
- feature: mean amplitude in selected time-window

In [25]:
tested_regressors = [
    (ln, ln_params),
]

regressor_params = dict()

In [26]:
results_pe_linear = pd.DataFrame()

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

this_bin=12

for scale in scales:
    print(f"---------------------SCALE: {scale}-------------------------")
    
    y_train = np.array(X_train_df[scale].to_list())
    y_test = np.array(X_test_df[scale].to_list())

    X_train_df_copy = pd.DataFrame(copy.deepcopy(X_train_df.to_dict()))
    X_test_df_copy = pd.DataFrame(copy.deepcopy(X_test_df.to_dict()))


    pipeline_name = f"ERP_scale_{scale}"

    ############################################################################################
    preprocessed_pipeline = Pipeline([
        ("channels_extraction",PickChannels(channels_list=['Cz'])),
        ("trim", EpochTrim(tmin=0.15, tmax=0.35)),
        ("average", Evoked()),
        ('extract_averaged_data', ExtractData()),
    ]).fit(X_train_df_copy)

    preprocessed_X = preprocessed_pipeline.transform(X_train_df_copy)


    ###########################################################################################
    features = Pipeline(steps = [
        ("mean_amplitude", AverageSignal()),
    ])

    steps = [('features', features)]

    ############################################################################################

    regressor_steps = steps

    # rate different models
    results_pe_linear = run_experiment_estimate_train_p(
        tested_regressors,
        regressor_params,
        pipeline_name,
        preprocessed_X,
        X_test_df_copy,
        y_train,
        y_test,
        dataset_name,
        regressor_steps,
        preprocessed_pipeline,
        X_test_df_copy,
        scale,
        results_pe_linear,
    )

---------------------SCALE: 16-Rumination Full Scale-------------------------
IN PE RETURN SHAPE: (206, 1, 53)
IN PE RETURN SHAPE: (206, 1, 53)
(96, 1, 53)
Rating ('ln', LinearRegression()) 

Fitting 3 folds for each of 1 candidates, totalling 3 fits
     Best parameters: {}
     mean r2: -0.014984995331746168           ± 0.017
     mean r2 train: 0.006242286090365738
     The permutation P-value is = 0.1558
     The permutation score is = -0.0150

IN PE RETURN SHAPE: (178, 1, 53)
(34, 1, 53)
score: 0.012189376667329777 
score: 0.005524611374958077 
     external validation r2: 0.012189376667329777      p-value:0.21878121878121878
None
---------------------SCALE: 05-DASS-21 Anxiety scale-------------------------
IN PE RETURN SHAPE: (206, 1, 53)
IN PE RETURN SHAPE: (206, 1, 53)
(96, 1, 53)
Rating ('ln', LinearRegression()) 

Fitting 3 folds for each of 1 candidates, totalling 3 fits
     Best parameters: {}
     mean r2: -0.053952221124350164           ± 0.016
     mean r2 train: 0.0172

In [28]:
results_pe_linear

,data_set,pipeline_name,model,parameters,mean_cv_r2,std_cv_r2,mean_cv_mae,std_cv_mae,mean_cv_mse,std_cv_mse,...,mean_train_mae,mean_train_mse,p-value,best_estimator,pre_processed_pipeline,scale,external_score,external_p-value,train_score,train_p-value
0,error_response,ERP_scale_16-Rumination Full Scale,ln,{},-0.014985,0.017166,-0.723096,0.047303,-0.810075,0.060208,...,-0.721832,-0.798068,0.155844,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",16-Rumination Full Scale,0.012189,0.218781,0.005525,0.230769
1,error_response,ERP_scale_05-DASS-21 Anxiety scale,ln,{},-0.053952,0.015895,-0.426781,0.021891,-0.286949,0.053698,...,-0.422897,-0.270468,0.473526,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",05-DASS-21 Anxiety scale,-0.106655,0.918082,0.011626,0.148851
2,error_response,ERP_scale_04-STAI Trait SUM,ln,{},-0.008557,0.006982,-7.964906,0.366396,-91.249981,9.153187,...,-7.950449,-90.416039,0.113886,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",04-STAI Trait SUM,0.004797,0.201798,0.001202,0.357642
3,error_response,ERP_scale_07-BIS,ln,{},-0.014374,0.010193,-0.495589,0.049433,-0.337214,0.056261,...,-0.490617,-0.332526,0.130869,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",07-BIS,-0.007908,0.964036,0.000070,0.468531
4,error_response,ERP_scale_14-Obsessive-Compulsive WASHING,ln,{},-0.069498,0.031129,-0.721684,0.051856,-0.766553,0.066373,...,-0.693539,-0.721923,0.617383,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",14-Obsessive-Compulsive WASHING,-0.059082,0.907093,0.003524,0.273726
5,error_response,ERP_scale_14-Obsessive-Compulsive OBSESSING,ln,{},-0.040845,0.042477,-0.937437,0.019957,-1.257335,0.095132,...,-0.918572,-1.187164,0.398601,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",14-Obsessive-Compulsive OBSESSING,0.057707,0.038961,0.012944,0.131868
6,error_response,ERP_scale_14-Obsessive-Compulsive HOARDING,ln,{},-0.078392,0.027651,-0.805741,0.150830,-0.989632,0.293556,...,-0.774602,-0.902674,0.645355,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",14-Obsessive-Compulsive HOARDING,-0.145110,0.749251,0.006488,0.216783
7,error_response,ERP_scale_14-Obsessive-Compulsive ORDERING,ln,{},-0.105803,0.075304,-1.044354,0.071047,-1.489871,0.177003,...,-0.993691,-1.368438,0.796204,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",14-Obsessive-Compulsive ORDERING,-0.003085,0.322677,0.000476,0.432567
8,error_response,ERP_scale_14-Obsessive-Compulsive CHECKING,ln,{},-0.060710,0.031793,-0.908193,0.021951,-1.179161,0.020398,...,-0.867288,-1.097729,0.528472,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",14-Obsessive-Compulsive CHECKING,-0.019563,0.531469,0.002130,0.335664
9,error_response,ERP_scale_14-Obsessive-Compulsive NEUTRALIZING,ln,{},-0.069743,0.054973,-0.479363,0.054679,-0.347015,0.085339,...,-0.466215,-0.326701,0.599401,"((AverageSignal()), LinearRegression())","(PickChannels(channels_list=['Cz']), EpochTrim...",14-Obsessive-Compulsive NEUTRALIZING,-0.056876,0.466533,0.016836,0.098901


In [34]:
results_pe_linear.to_csv("../public_data/results/models_results/Pe_waves_analysis_train_cv_lin.csv")

In [30]:
results_pe_linear.to_pickle("../public_data/results/models_pickles/pe_waves_analysis_train_cv_lin.pkl")

---
### Experiment 3: PCA-based analysis for ERN

- error responses;
- average over segments per subject;
- two ROIs;
- spatial PCA feature extraction;
- binning in 47ms-bins;
- centralization of the signal to the ERP peak for each participant
- time-window: two bins preceding the ERN peak to the one bin after the ERN peak; 
- feature: peak-to-peak amplitude in selected time-window

In [ ]:
# set-up

tested_regressors = [
    (svr, svr_params), 
    (kr, kr_params), 
    (en, en_params)
]

regressor_params = dict()

min_spatial_filter = 3
max_spatial_filter = 5
step_spatial_filter = 1


roi_1 = [
    "Fpz", 
    "AFz",
    "Fz",
    "FCz",
    "Cz",
    "CPz",
    "P1", "Pz", "P2",
]


roi_2 = [
    "Fpz", 
    "AFz",
    "F1", "Fz", "F2",
    "FCz",
    "C1", "Cz","C2",
    "CPz",
    "P1", "Pz", "P2",
]

roi_list = [roi_1, roi_2]

In [48]:
# wider signal, ERN: -2,1 Pe: 1,6
import copy

results_ern = pd.DataFrame()

# manually test different numbers of spatial filter components
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

this_bin=12

for scale in scales:
    print(f"---------------------SCALE: {scale}-------------------------")
    
    y_train = np.array(X_train_df[scale].to_list())
    y_test = np.array(X_test_df[scale].to_list())

    for roi in roi_list:

        X_train_df_copy = pd.DataFrame(copy.deepcopy(X_train_df.to_dict()))
        X_test_df_copy = pd.DataFrame(copy.deepcopy(X_test_df.to_dict()))

        print(f"----------BOX: {roi}")

        for n_components in range(min_spatial_filter, max_spatial_filter, step_spatial_filter): 
            print(f"---------------------SPATIAL FILTER : {n_components}-------------------------")

            pipeline_name = f"PCA_{n_components}_scale_{scale}"

            ############################################################################################
            preprocessed_pipeline = Pipeline([
                ("channels_extraction",PickChannels(channels_list=roi)),
                ("average", Evoked()),
                ('extract_averaged_data', ExtractData()),
                ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
                ("spatial_filter",PCA(n_components=n_components, random_state=random_state)),
                ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=timepoints_count)),
                ("lowpass_filter", LowpassFilter()),
                ("binning", BinTransformer(step=this_bin)),
                ("centering", CenteredSignalAfterBaseline3()) 

            ]).fit(X_train_df_copy)

            preprocessed_X = preprocessed_pipeline.transform(X_train_df_copy)

            ###########################################################################################

            ern_features = Pipeline(steps=[
                            ("ern_data_extraction", ErnTransformer()),
                            ("peak-to-peak", ErnAmplitude2()),
                            ("data_channel_swap", ChannelDataSwap()),
                            ("postprocessing", PostprocessingTransformer()),
                            ("scaler", StandardScaler()),
            ])

            ern_pe_features = FeatureUnion([
                ("ern_features", ern_features), 
            ], n_jobs = 10)

            steps = [('features', ern_pe_features)]

            ############################################################################################

            regressor_steps = steps

            # rate different models
            results_ern = run_experiment(
                tested_regressors,
                regressor_params,
                pipeline_name,
                preprocessed_X,
                X_test_df_copy,
                y_train,
                y_test,
                dataset_name,
                regressor_steps,
                preprocessed_pipeline,
                X_test_df_copy,
                scale,
                results_ern,
            )

In [ ]:
results_ern.to_pickle("../public_data/results/models_pickles/regression_union_100-600_cached_ern_amplitude_various_scales_with_external_p.pkl")

---
### Experiment 4: PCA-based analysis for Pe

- error responses;
- average over segments per subject;
- two ROIs;
- spatial PCA feature extraction;
- binning in 47ms-bins;
- centralization of the signal to the ERP peak for each participant
- time-window: from the first to the fifth bin after ERN peak; 
- feature: peak-to-peak amplitude in selected time-window

In [33]:
# set-up

tested_regressors = [
    (svr, svr_params), 
    (kr, kr_params), 
    (en, en_params)
]

regressor_params = dict()

min_spatial_filter = 3
max_spatial_filter = 5
step_spatial_filter = 1


roi_1 = [
    "Fpz", 
    "AFz",
    "Fz",
    "FCz",
    "C1", "Cz","C2",
    "CPz",
    "P1", "Pz", "P2",
]

roi_2 = [
    "Fpz",
    "AFz",
    "F1","Fz", "F2",
    "FC1", "FCz", "FC2",
    "C1","Cz","C2",
    "CP1", "CPz", "CP2",
    "P1","Pz", "P2",
]

roi_list = [roi_1, roi_2]

In [ ]:
results_pe = pd.DataFrame()

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

this_bin=12

for scale in scales:
    print(f"---------------------SCALE: {scale}-------------------------")
    
    y_train = np.array(X_train_df[scale].to_list())
    y_test = np.array(X_test_df[scale].to_list())

    for roi in roi_list:

        X_train_df_copy = pd.DataFrame(copy.deepcopy(X_train_df.to_dict()))
        X_test_df_copy = pd.DataFrame(copy.deepcopy(X_test_df.to_dict()))

        print(f"----------BOX: {roi}")

        for n_components in range(min_spatial_filter, max_spatial_filter, step_spatial_filter):
            print(f"---------------------SPATIAL FILTER : {n_components}-------------------------")

            pipeline_name = f"PCA_{n_components}_scale_{scale}"

            ############################################################################################
            preprocessed_pipeline = Pipeline([
                ("channels_extraction",PickChannels(channels_list=roi)),
                ("average", Evoked()),
                ('extract_averaged_data', ExtractData()),
                ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
                ("spatial_filter",PCA(n_components=n_components, random_state=random_state)),
                ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=timepoints_count)),
                ("lowpass_filter", LowpassFilter()),
                ("binning", BinTransformer(step=this_bin)),
                ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()) 

            ]).fit(X_train_df_copy)

            preprocessed_X = preprocessed_pipeline.transform(X_train_df_copy)

            ###########################################################################################

            pe_features = Pipeline(steps = [
                            ("pe_data_extraction", PeTransformer(3, 8)),
                            ("peak-to-peak", PeAmplitude2()),
                            ("data_channel_swap", ChannelDataSwap()),
                            ("postprocessing", PostprocessingTransformer()),
                            ("scaler", StandardScaler()),
            ])

            ern_pe_features = FeatureUnion([
                ("pe_features", pe_features)
            ], n_jobs = 10)

            steps = [('features', ern_pe_features)]

            ############################################################################################

            regressor_steps = steps

            # rate different models
            results_pe = run_experiment(
                tested_regressors,
                regressor_params,
                pipeline_name,
                preprocessed_X,
                X_test_df_copy,
                y_train,
                y_test,
                dataset_name,
                regressor_steps,
                preprocessed_pipeline,
                X_test_df_copy,
                scale,
                results_pe,
            )

In [ ]:
results_pe.to_pickle("../public_data/results/models_pickles/regression_union_100-600_cached_pe_amplitude_various_scales_with_external_p.pkl")